# Extracting SNP/P-value relations from tables

This module parses XML tables and extracts relations between SNPs and the p-values at which they are deemed to be significant.

## Preparations

We start by configuring Jupyter and setting up our environment.

In [3]:
%load_ext autoreload
%autoreload 2

import sys
import cPickle
import numpy as np
import sqlalchemy

# set the paths to snorkel and gwaskb
sys.path.append('../snorkel-tables')
sys.path.append('../src')
sys.path.append('../src/crawler')

# set up the directory with the input papers
supp_root_dir = '/Users/kuleshov/Downloads/gwas_supp2/'
supp_sub_dirs = ['doc/', 'docx/', 'xls/', 'xlsx/']
supp_dirs = [supp_root_dir + d for d in supp_sub_dirs]
supp_map_file = '../data/db/supp-map.txt'

# set up matplotlib
import matplotlib
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (12,4)

# create a Snorkel session
from snorkel import SnorkelSession
session = SnorkelSession()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Load corpus

We load our usual corpus of GWAS papers.

In [4]:
from extractor.parser import SuppXMLDocParser

xml_parser = SuppXMLDocParser(
    paths=supp_dirs,
    map_path=supp_map_file,
    doc='.//body',
    text='.//table',
    keep_xml_tree=True)

In [5]:
session.rollback()

In [8]:
from snorkel.models import Corpus
from snorkel.parser import CorpusParser, OmniParser

# parses tables into rows, cols, cells...
table_parser = OmniParser(timeout=1000000)

try:
    corpus = session.query(Corpus).filter(Corpus.name == 'GWAS Supplementary Table Corpus 13').one()
except:
    cp = CorpusParser(xml_parser, table_parser, max_docs=3)
    %time corpus = cp.parse_corpus(name='GWAS Supplementary Table Corpus 13', session=session)
    session.add(corpus)
    session.commit()

print 'Loaded corpus of %d documents' % len(corpus)

Loaded corpus of 3 documents


## Candidate Extraction

In [9]:
for d in corpus.documents:
    if d.tables: 
        for c in d.tables[0].cells[:50]:
            print c.text
            print c.phrases
#     if '1471-2261-11-29-s12' in d.name:
#         print d, d.meta['root'][:100]

<td>Principal Component</td>
[Phrase(Doc: 21223598-1755-8794-4-4-s2, Table: 0, Row: 0, Col: 0, Position: 0, Text: Principal Component)]
<td>Eigenvalue</td>
[Phrase(Doc: 21223598-1755-8794-4-4-s2, Table: 0, Row: 0, Col: 1, Position: 0, Text: Eigenvalue)]
<td>Relative Decrease</td>
[Phrase(Doc: 21223598-1755-8794-4-4-s2, Table: 0, Row: 0, Col: 2, Position: 0, Text: Relative Decrease)]
<td>r2 with Family</td>
[Phrase(Doc: 21223598-1755-8794-4-4-s2, Table: 0, Row: 0, Col: 3, Position: 0, Text: r2 with Family)]
<td>r2 with Center</td>
[Phrase(Doc: 21223598-1755-8794-4-4-s2, Table: 0, Row: 0, Col: 4, Position: 0, Text: r2 with Center)]
<td>1</td>
[Phrase(Doc: 21223598-1755-8794-4-4-s2, Table: 0, Row: 1, Col: 0, Position: 0, Text: 1)]
<td>7.511</td>
[Phrase(Doc: 21223598-1755-8794-4-4-s2, Table: 0, Row: 1, Col: 1, Position: 0, Text: 7.511)]
<td>1.13866742596811</td>
[Phrase(Doc: 21223598-1755-8794-4-4-s2, Table: 0, Row: 1, Col: 2, Position: 0, Text: 1.13866742596811)]
<td>0.0518</td>
[Phrase(

In [9]:
print corpus.documents[0].name

01471-2261-11-29-s1.html


In [5]:
import lxml.etree as et

In [13]:
d=et.parse('/Users/kuleshov/Downloads/gwas_supp2/doc/1471-2261-11-29-s12.html', et.HTMLParser()).xpath('.//table')

### Defining candidate matchers

We genereate RSid candidates from all spans that match the following regular expression.

In [10]:
from snorkel.matchers import RegexMatchSpan
rsid_matcher = RegexMatchSpan(rgx=r'rs\d+(/[ATCG]{1,2})*$')

Similarly, p-value candidates are all spans that match the following regular expression.

In [11]:
from snorkel.candidates import TableNgrams
from snorkel.matchers import RegexMatchSpan, Union

# 1: p-value matcher

rgx1 = u'[1-9]\d?[\xb7\.]?\d*[\s\u2009]*[\xd7\xb7\*][\s\u2009]*10[\s\u2009]*[-\u2212\u2013\u2012][\s\u2009]*\d+'
pval_rgx_matcher1 = RegexMatchSpan(rgx=rgx1)
rgx2 = u'[1-9]\d?[\xb7\.]?\d*[\s\u2009]*[eE][\s\u2009]*[-\u2212\u2013\u2012][\s\u2009]*\d+'
pval_rgx_matcher2 = RegexMatchSpan(rgx=rgx2)
rgx3 = u'0\.0000+\d+'
pval_rgx_matcher3 = RegexMatchSpan(rgx=rgx3)
pval_rgx_matcher = Union(pval_rgx_matcher1, pval_rgx_matcher2, pval_rgx_matcher3)

# 2: column-based matcher (currently not used)

from snorkel.matchers import CellNameRegexMatcher

pval_rgx = 'p\s?.?\s?value'
pval_rgxname_matcher = CellNameRegexMatcher(axis='col', rgx=pval_rgx, n_max=3, ignore_case=True, header_only=True, max_chars=20)

# 3: combine the two

pval_matcher = Union(pval_rgx_matcher, pval_rgxname_matcher)

## Extract candidate relations between SNPs and p-values

In [12]:
# create a Snorkel class for the relation we will extract
from snorkel.models import candidate_subclass
RsidPhenRel = candidate_subclass('RsidPvalRel', ['rsid','pval'])

# define our candidate spaces
from snorkel.candidates import TableNgrams
unigrams = TableNgrams(n_max=1)
heptagrams = TableNgrams(n_max=7)

# we will be looking only at aligned cells
from snorkel.throttlers import AlignmentThrottler
row_align_filter = AlignmentThrottler(axis='row', infer=False)

# the first extractor looks at phenotype names in columns with a header indicating it's a phenotype
from snorkel.candidates import CandidateExtractor
ce = CandidateExtractor(RsidPhenRel, [unigrams, heptagrams], [rsid_matcher, pval_rgx_matcher], throttler=row_align_filter)

# collect that cells that will be searched for candidates
tables = [table for doc in corpus.documents for table in doc.tables]

In [18]:
from snorkel.models import CandidateSet
    
try:
    rels = session.query(CandidateSet).filter(CandidateSet.name == 'RsidPvalRel Relations 2').one()
except:
    %time rels = ce.extract(tables, 'RsidPvalRel Relations 2', session)
    session.add(rels)
    session.commit()

print "%s relations extracted, e.g." % len(rels)
for cand in rels[:10]:
    print cand

495 relations extracted, e.g.
RsidPvalRel(Span("rs2243504", parent=35607, chars=[0,8], words=[0,0]), Span("0.00002165404", parent=35610, chars=[0,12], words=[0,0]))
RsidPvalRel(Span("rs9825324", parent=36167, chars=[0,8], words=[0,0]), Span("0.00009167234", parent=36170, chars=[0,12], words=[0,0]))
RsidPvalRel(Span("rs6894496", parent=35639, chars=[0,8], words=[0,0]), Span("0.00002495544", parent=35642, chars=[0,12], words=[0,0]))
RsidPvalRel(Span("rs2910597", parent=36103, chars=[0,8], words=[0,0]), Span("0.00008622475", parent=36106, chars=[0,12], words=[0,0]))
RsidPvalRel(Span("rs9296092", parent=35999, chars=[0,8], words=[0,0]), Span("0.00006919188", parent=36002, chars=[0,12], words=[0,0]))
RsidPvalRel(Span("rs166315", parent=36071, chars=[0,7], words=[0,0]), Span("0.00007825004", parent=36074, chars=[0,12], words=[0,0]))
RsidPvalRel(Span("rs12656156", parent=35951, chars=[0,9], words=[0,0]), Span("0.00006416992", parent=35954, chars=[0,12], words=[0,0]))
RsidPvalRel(Span("rs39125

#### Save this for now

In [19]:
import re
from extractor.util import pvalue_to_float

def clean_rsid(rsid):
    return re.sub('/.+', '', rsid)

with open('results/nb-output/supp-pval-rsid.tsv', 'w') as f:
    for rel in rels:
        docname = rel[0].parent.document.name
        pmid = docname.split('-')[0]
        table_id = rel[0].parent.table.position
        row_id = rel[0].parent.cell.row.position
        col_id = rel[0].parent.cell.col.position
        rsid = rel[0].get_span()
        log_pval = pvalue_to_float(rel[1].get_span())
        
        try:
            out_str = '%s\t%s\t%d\t%d\t%d\t%f\n' % (pmid, clean_rsid(rsid), table_id, row_id, col_id, log_pval)
        except:
            print 'could not write:', pmid, clean_rsid(rsid), table_id, row_id, col_id, log_pval
        f.write(out_str)

## Extracting singleton SNPs

There also exist many papers that don't report p-values. To handle these, we also report certain rsids that have *not* been associated with a p-value.

Here, we extract these entites. Later on, we will filter them.

In [12]:
# Define the extractor
from snorkel.models import candidate_subclass
from snorkel.matchers import RegexMatchSpan
from snorkel.candidates import CandidateExtractor

RSID = candidate_subclass('SnorkelRsid 2', ['rsid'])

unigrams = TableNgrams(n_max=1)
rsid_singleton_matcher = RegexMatchSpan(rgx=r'rs\d+(/[^s]+)?')
rsid_singleton_extractor = CandidateExtractor(RSID, unigrams, rsid_singleton_matcher)

Perform the extraction process.

In [13]:
from snorkel.models import CandidateSet

try:
    rsid_c = session.query(CandidateSet).filter(CandidateSet.name == 'Rsid Candidates 2').one()
except:
    tables = [table for doc in corpus.documents for table in doc.tables]
    print '%d tables loaded' % len(tables)
    %time rsid_c = rsid_singleton_extractor.extract(tables, 'Rsid Candidates 2', session)
    session.add(rsid_c)
    session.commit()

print '%d candidates extracted' % len(rsid_c)

1648 tables loaded
[========================================] 100%
CPU times: user 3min 31s, sys: 1min 13s, total: 4min 44s
Wall time: 4min 48s
15333 candidates extracted


We store candidates that occur in sufficiently large tables:

In [14]:
rsid_by_table = dict()
for cand in rsid_c:
    rsid = cand[0].get_span()
    key = cand[0].parent.document.name, cand[0].parent.table.position
    if key not in rsid_by_table: rsid_by_table[key] = set()
    rsid_by_table[key].add((rsid, cand[0].parent.cell.row.position, cand[0].parent.cell.col.position))
    
with open('results/nb-output/rsids.singletons.all.tsv', 'w') as f:
    for (pmid, table_id), rsids in rsid_by_table.items():
        if len(rsids) < 10: continue
        for rsid, row_num, col_num in rsids:
            f.write('%s\t%s\t%s\t%s\t%s\n' % (pmid, table_id, row_num, col_num, rsid))

Here, we store certain table features that will be used to select which singelton rsid's to report.

In [11]:
import re
from bs4 import BeautifulSoup as soup

pval_rgx = 'p\s?.?\s?value'
lod_rgx = 'LOD'

def has_pval(txt):
    if re.search(pval_rgx, txt, re.IGNORECASE):
        return True
    elif txt.lower() == 'p':
        return True
    return False

with open('results/nb-output/table-annotations.tsv', 'w') as f:
    for doc in corpus.documents:
        for table in doc.tables:
            lod_found = 0
            pval_found = 0
            for cell in table.cells:
                txt = soup(cell.text).text
                if not pval_found and len(txt) < 30 and has_pval(txt):
                    pval_found = 1
                if not lod_found and re.search(lod_rgx, txt):
                    lod_found = 1
                if pval_found and lod_found: break
                    
            out_str = '%s\t%s\t%s\t%s\n' % (doc.name, table.position, pval_found, lod_found)
            f.write(out_str) 

/Library/Python/2.7/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 162 of the file /System/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


## Filtering relations

Here, we perform a bit of filtering in post-processing.

In [20]:
rels = []
loc2rsid = dict()
with open('results/nb-output/supp-pval-rsid.tsv') as f:
    for line in f:
        pmid, rsid, table_id, row_id, col_id, pval = line.strip().split('\t')
        loc = pmid, table_id, row_id
        rels.append((pmid, rsid, table_id, row_id, col_id, pval))
        if loc not in loc2rsid: loc2rsid[loc] = set()
        loc2rsid[loc].add(rsid)

n = 0
with open('results/nb-output/supp-pval-rsid.filtered.tsv', 'w') as f:
    for rel in rels:
        pmid, rsid, table_id, row_id, col_id, pval = rel
        loc = pmid, table_id, row_id
        if len(loc2rsid[loc]) > 1: continue
        
        out_str = '%s\t%s\t%s\t%s\t%s\t%s\n' % (pmid, rsid, table_id, row_id, col_id, pval)
        f.write(out_str)
        n += 1
        
print len(rels), n

495 495
